In [1]:
from numpy import loadtxt
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score #not used?
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import MinMaxScaler
import datetime
import os
import matplotlib.pylab as plt
import time


%matplotlib inline

D:\DProgramFiles\python3Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
DATA_DIR = "D:/2017_working/unearthed/output/"
INPUT_DATA_DIR = "D:/2017_working/unearthed/"
TARGET = "target"
NORM_TARGET = "norm_target"
UNAMED = "Unnamed: 0"
TIMESTAMP = "timestamp"

In [3]:
df_test = pd.read_csv(INPUT_DATA_DIR+'publishable_test_set.csv')
print (type(df_test), df_test.shape)

<class 'pandas.core.frame.DataFrame'> (439140, 27)


D:\DProgramFiles\python3Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
colnames = list(df_test)
colnames

['timestamp',
 'WQI8100XCL1.CPV',
 'XI84201.PV',
 'XI84202.PV',
 'XI84123.PV',
 'XI84124.PV',
 'XI84125.PV',
 'FX87211.CPV1',
 'FIC87211.PV',
 'FIC87211.SV',
 'FX87211.P01',
 'FI87208.PV',
 'AIC88049.PV',
 'ZI88001.PV',
 'NIC88002.PV',
 'PIC88007.PV',
 'LIC88006.PV',
 'AIC88055.PV',
 'FIC88022.PV',
 'II88151.PV',
 'II88152.PV',
 'SI88033.PV',
 'SI88034.PV',
 'MQI88024.CPV',
 'FV88156.PV',
 'FV88043.PV',
 'FV88044.PV']

In [5]:
#convert df[TIMESTAMP] from string to datetime
df_test[TIMESTAMP] = pd.to_datetime(df_test[TIMESTAMP])

In [6]:
df_test.dtypes

timestamp          datetime64[ns]
WQI8100XCL1.CPV            object
XI84201.PV                 object
XI84202.PV                 object
XI84123.PV                 object
XI84124.PV                 object
XI84125.PV                 object
FX87211.CPV1               object
FIC87211.PV                object
FIC87211.SV                object
FX87211.P01                object
FI87208.PV                 object
AIC88049.PV                object
ZI88001.PV                 object
NIC88002.PV                object
PIC88007.PV                object
LIC88006.PV                object
AIC88055.PV                object
FIC88022.PV                object
II88151.PV                 object
II88152.PV                 object
SI88033.PV                 object
SI88034.PV                 object
MQI88024.CPV               object
FV88156.PV                 object
FV88043.PV                 object
FV88044.PV                 object
dtype: object

In [7]:
print (type(df_test[TIMESTAMP].iloc[0]), df_test[TIMESTAMP].iloc[0])

<class 'pandas._libs.tslib.Timestamp'> 2016-05-01 00:00:00


In [8]:
print (df_test[TIMESTAMP].iloc[0], df_test[TIMESTAMP].dtypes, type(df_test[TIMESTAMP].iloc[0]))

time_step = (df_test[TIMESTAMP].iloc[1]-df_test[TIMESTAMP].iloc[0])
print ("time_step(in seconds):", time_step, type(time_step))
time_start = df_test[TIMESTAMP].iloc[0]

print ("time_start = df[TIMESTAMP][0]", type(df_test[TIMESTAMP].iloc[0]), df_test[TIMESTAMP].iloc[0])

2016-05-01 00:00:00 datetime64[ns] <class 'pandas._libs.tslib.Timestamp'>
time_step(in seconds): 0 days 00:01:00 <class 'pandas._libs.tslib.Timedelta'>
time_start = df[TIMESTAMP][0] <class 'pandas._libs.tslib.Timestamp'> 2016-05-01 00:00:00


In [9]:
#convert datetime to int 
#convert datetime to int since XGBoost does not handle datetime
#NB: df[TIMESTAMP][0] is set to zero with subsequent values = zero + seconds.
print ("converting df_test[TIMESTAMP] to int")
timestampNew = "timestamp_"
df_test[timestampNew] = df_test.apply(lambda x: int((x[TIMESTAMP] - time_start).total_seconds()), axis=1)
print ("converting df_test[TIMESTAMP] to int: completed.")
df_test.drop(labels=TIMESTAMP, axis=1, inplace=True)
df_test.rename(index=str, columns={timestampNew: TIMESTAMP}, inplace=True)

#NB: timestamp is now last column in dataframe

converting df_test[TIMESTAMP] to int
converting df_test[TIMESTAMP] to int: completed.


In [10]:
df_test[TIMESTAMP].dtypes, type(df_test[TIMESTAMP]), df_test[TIMESTAMP].iloc[0], df_test[TIMESTAMP].iloc[1]

(dtype('int64'), pandas.core.series.Series, 0, 60)

In [11]:
null_count = df_test.isnull().sum()
print ("null_count:\n", null_count)
print ("total nulls:", null_count.sum())


null_count:
 WQI8100XCL1.CPV    0
XI84201.PV         0
XI84202.PV         0
XI84123.PV         0
XI84124.PV         0
XI84125.PV         0
FX87211.CPV1       0
FIC87211.PV        0
FIC87211.SV        0
FX87211.P01        0
FI87208.PV         0
AIC88049.PV        0
ZI88001.PV         0
NIC88002.PV        0
PIC88007.PV        0
LIC88006.PV        0
AIC88055.PV        0
FIC88022.PV        0
II88151.PV         0
II88152.PV         0
SI88033.PV         0
SI88034.PV         0
MQI88024.CPV       0
FV88156.PV         0
FV88043.PV         0
FV88044.PV         0
timestamp          0
dtype: int64
total nulls: 0


In [12]:
    bad_list = ['No Data', 'I/O Timeout', 'Bad Input', 'Scan Off']

In [13]:
print (df_test.shape)
cols_to_delete = []
for colname in colnames[1:]:
    #NB: TIMESTAMP column already converted above.
    try:
        badcount = []
        for bad in bad_list:
            badcount.append(df_test[colname].str.contains(bad).sum())
        print (colname, "\t", dict(zip(bad_list, badcount))  , df_test[colname].dtype)
        if badcount[0] == df_test.shape[0]:
            cols_to_delete.append(colname)
    except Exception as e: 
        print(e)
        print ("error", colname)
        #added this since reusing this code block for data exploration
print ("cols_to_delete:", cols_to_delete)


(439140, 27)
WQI8100XCL1.CPV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 18, 'Scan Off': 0} object
XI84201.PV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 19, 'Scan Off': 0} object
XI84202.PV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 19, 'Scan Off': 0} object
XI84123.PV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 19, 'Scan Off': 0} object
XI84124.PV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 26, 'Scan Off': 0} object
XI84125.PV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 26, 'Scan Off': 0} object
FX87211.CPV1 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 18, 'Scan Off': 0} object
FIC87211.PV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 18, 'Scan Off': 0} object
FIC87211.SV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 18, 'Scan Off': 0} object
FX87211.P01 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 18, 'Scan Off': 0} object
FI87208.PV 	 {'No Data': 0, 'I/O Timeout': 0, 'Bad Input': 18, 'Scan Off': 0} object
AIC88049.PV 	 {'No Data': 0, 'I/O Timeout'

In [14]:
print (df_test.shape)
df_test = df_test.drop(cols_to_delete, axis=1)
print (df_test.shape)

(439140, 27)
(439140, 22)


In [15]:
colnames = list(df_test)
colnames

['WQI8100XCL1.CPV',
 'XI84201.PV',
 'XI84202.PV',
 'XI84123.PV',
 'XI84124.PV',
 'XI84125.PV',
 'FX87211.CPV1',
 'FIC87211.PV',
 'FIC87211.SV',
 'FX87211.P01',
 'FI87208.PV',
 'AIC88049.PV',
 'ZI88001.PV',
 'NIC88002.PV',
 'PIC88007.PV',
 'LIC88006.PV',
 'AIC88055.PV',
 'FIC88022.PV',
 'SI88033.PV',
 'SI88034.PV',
 'MQI88024.CPV',
 'timestamp']

In [16]:
#test converting columns from string values to floats with errors going to NaN
for colname in colnames[:-1]:
    print ("converting column:", colname)
    df_test[colname] = pd.to_numeric(df_test[colname], errors='coerce')

converting column: WQI8100XCL1.CPV
converting column: XI84201.PV
converting column: XI84202.PV
converting column: XI84123.PV
converting column: XI84124.PV
converting column: XI84125.PV
converting column: FX87211.CPV1
converting column: FIC87211.PV
converting column: FIC87211.SV
converting column: FX87211.P01
converting column: FI87208.PV
converting column: AIC88049.PV
converting column: ZI88001.PV
converting column: NIC88002.PV
converting column: PIC88007.PV
converting column: LIC88006.PV
converting column: AIC88055.PV
converting column: FIC88022.PV
converting column: SI88033.PV
converting column: SI88034.PV
converting column: MQI88024.CPV


In [17]:
#count NaN values in each column, then replace
null_count = df_test.isnull().sum()
print ("null_count:\n", null_count)
print ("total nulls:", null_count.sum())


null_count:
 WQI8100XCL1.CPV    18
XI84201.PV         19
XI84202.PV         19
XI84123.PV         19
XI84124.PV         26
XI84125.PV         26
FX87211.CPV1       18
FIC87211.PV        18
FIC87211.SV        18
FX87211.P01        18
FI87208.PV         18
AIC88049.PV        18
ZI88001.PV         18
NIC88002.PV        18
PIC88007.PV        18
LIC88006.PV        18
AIC88055.PV        53
FIC88022.PV        18
SI88033.PV         18
SI88034.PV         18
MQI88024.CPV       18
timestamp           0
dtype: int64
total nulls: 432


In [18]:
#convert null values to floats by filling data.
#df_test.fillna(method='bfill', axis=1, inplace=True)
for colname in colnames:
    print ("filling colname:", colname)
    df_test[colname].fillna(method='bfill', inplace=True)
#NB: weird error if attempt fillna on dataframe. refer github unfixed error.

filling colname: WQI8100XCL1.CPV
filling colname: XI84201.PV
filling colname: XI84202.PV
filling colname: XI84123.PV
filling colname: XI84124.PV
filling colname: XI84125.PV
filling colname: FX87211.CPV1
filling colname: FIC87211.PV
filling colname: FIC87211.SV
filling colname: FX87211.P01
filling colname: FI87208.PV
filling colname: AIC88049.PV
filling colname: ZI88001.PV
filling colname: NIC88002.PV
filling colname: PIC88007.PV
filling colname: LIC88006.PV
filling colname: AIC88055.PV
filling colname: FIC88022.PV
filling colname: SI88033.PV
filling colname: SI88034.PV
filling colname: MQI88024.CPV
filling colname: timestamp


In [19]:
#count NaN values in each column, then replace
null_count = df_test.isnull().sum()
print ("null_count:\n", null_count)
print ("total nulls:", null_count.sum())


null_count:
 WQI8100XCL1.CPV    0
XI84201.PV         0
XI84202.PV         0
XI84123.PV         0
XI84124.PV         0
XI84125.PV         0
FX87211.CPV1       0
FIC87211.PV        0
FIC87211.SV        0
FX87211.P01        0
FI87208.PV         0
AIC88049.PV        0
ZI88001.PV         0
NIC88002.PV        0
PIC88007.PV        0
LIC88006.PV        0
AIC88055.PV        0
FIC88022.PV        0
SI88033.PV         0
SI88034.PV         0
MQI88024.CPV       0
timestamp          0
dtype: int64
total nulls: 0


In [20]:
output_file = DATA_DIR+"test_set_cleaned.csv"
print ("output_file:", output_file)
df_test.to_csv(path_or_buf=output_file)


output_file: D:/2017_working/unearthed/output/test_set_cleaned.csv


In [21]:
#now create new column with average over 5 minute blocks
#first 5-1 minute blocks will be filled using forward five minute average otherwise algo will not generate.

In [22]:
#create new empty columns for averages.
for colname in colnames[:-1]:
    print ("colname:", colname)
    df_test[colname+'_5'] = np.nan
print ("colnames:", list(df_test))

colname: WQI8100XCL1.CPV
colname: XI84201.PV
colname: XI84202.PV
colname: XI84123.PV
colname: XI84124.PV
colname: XI84125.PV
colname: FX87211.CPV1
colname: FIC87211.PV
colname: FIC87211.SV
colname: FX87211.P01
colname: FI87208.PV
colname: AIC88049.PV
colname: ZI88001.PV
colname: NIC88002.PV
colname: PIC88007.PV
colname: LIC88006.PV
colname: AIC88055.PV
colname: FIC88022.PV
colname: SI88033.PV
colname: SI88034.PV
colname: MQI88024.CPV
colnames: ['WQI8100XCL1.CPV', 'XI84201.PV', 'XI84202.PV', 'XI84123.PV', 'XI84124.PV', 'XI84125.PV', 'FX87211.CPV1', 'FIC87211.PV', 'FIC87211.SV', 'FX87211.P01', 'FI87208.PV', 'AIC88049.PV', 'ZI88001.PV', 'NIC88002.PV', 'PIC88007.PV', 'LIC88006.PV', 'AIC88055.PV', 'FIC88022.PV', 'SI88033.PV', 'SI88034.PV', 'MQI88024.CPV', 'timestamp', 'WQI8100XCL1.CPV_5', 'XI84201.PV_5', 'XI84202.PV_5', 'XI84123.PV_5', 'XI84124.PV_5', 'XI84125.PV_5', 'FX87211.CPV1_5', 'FIC87211.PV_5', 'FIC87211.SV_5', 'FX87211.P01_5', 'FI87208.PV_5', 'AIC88049.PV_5', 'ZI88001.PV_5', 'NIC880

In [23]:
df_test.shape

(439140, 43)

In [24]:
colnames.index(TIMESTAMP)

21

In [25]:
colnames[0:colnames.index(TIMESTAMP)]

['WQI8100XCL1.CPV',
 'XI84201.PV',
 'XI84202.PV',
 'XI84123.PV',
 'XI84124.PV',
 'XI84125.PV',
 'FX87211.CPV1',
 'FIC87211.PV',
 'FIC87211.SV',
 'FX87211.P01',
 'FI87208.PV',
 'AIC88049.PV',
 'ZI88001.PV',
 'NIC88002.PV',
 'PIC88007.PV',
 'LIC88006.PV',
 'AIC88055.PV',
 'FIC88022.PV',
 'SI88033.PV',
 'SI88034.PV',
 'MQI88024.CPV']

In [26]:
start = time.time()
nrows = 5

for colname in colnames[:colnames.index(TIMESTAMP)]:
    print ("processing colname:", colname)
    for i in range(0, nrows):
        #print ("at i = ", i, "\n", df_test[colname].iloc[i])
        avg_ = df_test[colname].iloc[i:i+nrows].mean()
        #print ("colname:", colname, "type(avg_):", type(avg_), avg_)
        df_test[colname+"_5"].iloc[i] = avg_

end = time.time()
print("time to average first ", nrows, " rows:", end - start)


processing colname: WQI8100XCL1.CPV


D:\DProgramFiles\python3Anaconda\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


processing colname: XI84201.PV
processing colname: XI84202.PV
processing colname: XI84123.PV
processing colname: XI84124.PV
processing colname: XI84125.PV
processing colname: FX87211.CPV1
processing colname: FIC87211.PV
processing colname: FIC87211.SV
processing colname: FX87211.P01
processing colname: FI87208.PV
processing colname: AIC88049.PV
processing colname: ZI88001.PV
processing colname: NIC88002.PV
processing colname: PIC88007.PV
processing colname: LIC88006.PV
processing colname: AIC88055.PV
processing colname: FIC88022.PV
processing colname: SI88033.PV
processing colname: SI88034.PV
processing colname: MQI88024.CPV
time to average first  5  rows: 3.088268995285034


In [ ]:
#TODO: check how to speed up next block. not performant.
#possibly lambda function???

In [ ]:
start = time.time()

for colname in colnames[:colnames.index(TIMESTAMP)]:
    start2 = time.time()

    print ("processing colname:", colname)
    for i in range(nrows, df_test.shape[0]):
        #print ("at i = ", i, "\n", df_test.iloc[i])
        avg_ = df_test[colname].iloc[i-nrows+1:i+1].mean()
        #print ("type(avg_):", type(avg_))
        df_test[colname+"_5"].iloc[i] = avg_
        #if i==100: break  
        if i%1000==0: 
            print ("i:", i, ", elapsed time:", time.time()-start2)
    end2 = time.time()
    print("time to average:", end2 - start2)

end = time.time()
print("time to average first ", nrows, " rows:", end - start)


processing colname: WQI8100XCL1.CPV


D:\DProgramFiles\python3Anaconda\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


i: 1000 , elapsed time: 28.31411576271057
i: 2000 , elapsed time: 64.76399278640747
i: 3000 , elapsed time: 93.26177978515625
i: 4000 , elapsed time: 126.2707109451294
i: 5000 , elapsed time: 153.5442454814911
i: 6000 , elapsed time: 180.47986793518066
i: 7000 , elapsed time: 207.41249322891235
i: 8000 , elapsed time: 235.60847854614258
i: 9000 , elapsed time: 267.511536359787
i: 10000 , elapsed time: 298.5911531448364
i: 11000 , elapsed time: 334.05772280693054
i: 12000 , elapsed time: 367.2577586174011
i: 13000 , elapsed time: 403.4783413410187
i: 14000 , elapsed time: 431.5377821922302
i: 15000 , elapsed time: 461.7913854122162
i: 16000 , elapsed time: 490.89760994911194
i: 17000 , elapsed time: 519.1046214103699
i: 18000 , elapsed time: 547.0620152950287
i: 19000 , elapsed time: 575.2890801429749
i: 20000 , elapsed time: 607.2349545955658
i: 21000 , elapsed time: 639.8145968914032
i: 22000 , elapsed time: 668.4593245983124
i: 23000 , elapsed time: 699.5554258823395
i: 24000 , elaps

In [ ]:
df_test.head(6)

In [ ]:
df_test.head(110)

In [ ]:
output_file = DATA_DIR+"test_set_cleaned_norm_5.csv"
print ("output_file:", output_file)
df_test.to_csv(path_or_buf=output_file)
